In [1]:
import numpy as np 
import pandas as pd
import re
from datetime import datetime
import time
import warnings

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
df = pd.read_csv("/home/jbloch3/Documents/Proyecto_TL//attacks.csv")

**Observamos el contenido de las columnas del DataFrame**

In [3]:
#df.describe

In [4]:
df.head(5)

df.columns
    

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

**Adecuamos los nombres de las columnas para poder manipularlas con facilidad**

In [5]:

df = df.rename(columns={k:v for k,v in [('Unnamed: 22','Unnamed22'), ('Species ', 'Species'), ('Sex ', 'Sex'), ('Investigator or Source', 'Investigator_Source'), ('Unnamed: 23', 'Unnamed23'), ('Fatal (Y/N)', 'Fatal'), ('original order', 'Original_order'), ('href formula', 'Href_formula'), ('href', 'Href'), ('Case Number', 'Case_Number'), ('Case Number.1', 'Case_Number.1'), ('Case Number.2', 'Case_Number.2')]})
# cambiamos los nombres de las columnas con un diccionario para no tener problemas al subsetear

In [6]:
df.columns

Index(['Case_Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species',
       'Investigator_Source', 'pdf', 'Href_formula', 'Href', 'Case_Number.1',
       'Case_Number.2', 'Original_order', 'Unnamed22', 'Unnamed23'],
      dtype='object')

**Guardamos la dimensión original del DF**

In [7]:
dim_original=df.shape
df.shape

(25723, 24)

**Cambiamos las casillas vacías por na para facilitar el tratamiento**

In [8]:
df.replace("", np.nan, inplace=True)
#cambiamos las casillas con strings vaciás por na

**Borramos todas las filas con mas de 3 fallos**

In [9]:
df = df.dropna(0, thresh=3)

In [10]:
df.shape

(6302, 24)

In [11]:
df

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator_Source,pdf,Href_formula,Href,Case_Number.1,Case_Number.2,Original_order,Unnamed22,Unnamed23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN,NaN
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN,NaN
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN,NaN
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8�N, 79�W",NaN,Jules Patterson,M,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN,NaN


**Observamos los valores na por columnas**

In [12]:
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

Case_Number               1
Year                      2
Type                      4
Country                  50
Area                    455
Location                540
Activity                544
Name                    210
Sex                     565
Age                    2831
Injury                   28
Fatal                   539
Time                   3354
Species                2838
Investigator_Source      17
Href_formula              1
Unnamed22              6301
Unnamed23              6300
dtype: int64

In [13]:
df.dtypes

Case_Number             object
Date                    object
Year                   float64
Type                    object
Country                 object
Area                    object
Location                object
Activity                object
Name                    object
Sex                     object
Age                     object
Injury                  object
Fatal                   object
Time                    object
Species                 object
Investigator_Source     object
pdf                     object
Href_formula            object
Href                    object
Case_Number.1           object
Case_Number.2           object
Original_order         float64
Unnamed22               object
Unnamed23               object
dtype: object

**Buscamos los valores no na de la columna Unnamed22**

In [14]:
bad_index=df.Unnamed22[df.Unnamed22.notna()].index

bad_index

Int64Index([1478], dtype='int64')

In [15]:
df.loc[1478,]

Case_Number                                                   2006.05.27
Date                                                         27-May-2006
Year                                                              2006.0
Type                                                          Unprovoked
Country                                                              USA
Area                                                              Hawaii
Location                                              North Shore, O'ahu
Activity                                                         Surfing
Name                                                        Bret Desmond
Sex                                                                    M
Age                                                                   31
Injury                                 No injury, shark bumped surfboard
Fatal                                                                  N
Time                                               

**Usando set(df.Type) vemos en que columnas podemos cambiar na por unknown**

In [16]:
#
na_to_ukwn = ['Species', 'Investigator_Source', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Injury', 'Type', 'Href_formula']



**Rellenamos por Unknown**

In [17]:
for i in na_to_ukwn:
    df[i]=df[i].fillna('Unknown')



In [18]:
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

Case_Number       1
Year              2
Sex             565
Age            2831
Fatal           539
Time           3354
Unnamed22      6301
Unnamed23      6300
dtype: int64

**Normalizamos la variable Sex**

In [19]:

set(df.Sex)

{'.', 'F', 'M', 'M ', 'N', 'lli', nan}

In [20]:
for i in range(len(df)):
    if str(df.Sex[i]).startswith('M'):
        df.Sex[i] = 'M'
    elif str(df.Sex[i]).startswith('F'):
        df.Sex[i] = 'F'
    else:
        df.Sex[i] = 'Unknown'
        
        

In [21]:
set(df.Sex)

{'F', 'M', 'Unknown'}

In [22]:
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

Case_Number       1
Year              2
Age            2831
Fatal           539
Time           3354
Unnamed22      6301
Unnamed23      6300
dtype: int64

**Normalizamos la variable Age**

In [23]:
#Buscada edad media ataques de tiburón(29 años)
lst= []
patron = '\d+'
for e in df.Age:
    try:
        lst.append(int(re.findall(patron,e)[0]))
    except:
        lst.append(int(29))

df.Age=lst    

In [24]:
print(set(df.Age))

{1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 81, 82, 84, 86, 87}


In [25]:
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

Case_Number       1
Year              2
Fatal           539
Time           3354
Unnamed22      6301
Unnamed23      6300
dtype: int64

**Normalizamos la variable Year**

In [26]:
#buscando las filas con año na, sustituimos por el año de la fecha
bad_index=df.Year[df.Year.isna()].index
bad_index

Int64Index([187, 6079], dtype='int64')

In [27]:
df.loc[(187,6079),] #las mostramos

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator_Source,pdf,Href_formula,Href,Case_Number.1,Case_Number.2,Original_order,Unnamed22,Unnamed23
187,2017.01.08.R,Reported 08-Jan-2017,NaN,Invalid,AUSTRALIA,Queensland,Unknown,Spearfishing,Kerry Daniel,M,...,Bull shark,Liquid Vision 1/8/2017,2017.01.08.R-KerryDaniel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.01.08.R,2017.01.08.R,6116.0,NaN,NaN
6079,1836.08.19.R,Reported 19-Aug-1836,NaN,Unprovoked,ENGLAND,Cumberland,Whitehaven,Swimming,a boy,M,...,Unknown,"C. Moore, GSAF",1835.08.19.R-Whitehaven.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1836.08.19.R,1836.08.19.R,224.0,NaN,NaN


In [28]:
df.Year[187] = (2017) #Las cambiamos por el ano de Date
df.Year[6079] = int(1836)

In [29]:
df.loc[(187,6079),]

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator_Source,pdf,Href_formula,Href,Case_Number.1,Case_Number.2,Original_order,Unnamed22,Unnamed23
187,2017.01.08.R,Reported 08-Jan-2017,2017.0,Invalid,AUSTRALIA,Queensland,Unknown,Spearfishing,Kerry Daniel,M,...,Bull shark,Liquid Vision 1/8/2017,2017.01.08.R-KerryDaniel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.01.08.R,2017.01.08.R,6116.0,NaN,NaN
6079,1836.08.19.R,Reported 19-Aug-1836,1836.0,Unprovoked,ENGLAND,Cumberland,Whitehaven,Swimming,a boy,M,...,Unknown,"C. Moore, GSAF",1835.08.19.R-Whitehaven.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1836.08.19.R,1836.08.19.R,224.0,NaN,NaN


In [30]:
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

Case_Number       1
Fatal           539
Time           3354
Unnamed22      6301
Unnamed23      6300
dtype: int64

**Normalizamos la variable Unnamed23**

In [31]:
bad_index=df.Unnamed23[df.Unnamed23.notna()].index
bad_index

Int64Index([4415, 5840], dtype='int64')

In [32]:
df.loc[(4415, 5840),]

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator_Source,pdf,Href_formula,Href,Case_Number.1,Case_Number.2,Original_order,Unnamed22,Unnamed23
4415,1952.03.30,30-Mar-1952,1952.0,Unprovoked,NETHERLANDS ANTILLES,Curacao,Unknown,Went to aid of child being menaced by the shark,A.J. Eggink,M,...,"Bull shark, 2.7 m [9'] was captured & dragged ...","J. Randall, p.352 in Sharks & Survival; H.D. B...",1952.03.30-Eggink.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.03.30,1952.03.30,1888.0,NaN,Teramo
5840,1878.09.14.R,Reported 14-Sep-1878,1878.0,Provoked,USA,Connecticut,"Branford, New Haven County",Fishing,Captain Pattison,M,...,Unknown,"St. Joseph Herald, 9/14/1878",1878.09.14.R-Pattison.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1878.09.14.R,1878.09.14.R,463.0,NaN,change filename


**Los rellenamos con 0 con los na**

In [33]:
df.Unnamed23[4415] = 0
df.Unnamed23[5840] = 0
df.Unnamed23=df.Unnamed23.fillna(0)

**Igual con Unnamed22**

In [34]:
bad_index2=df.Unnamed22[df.Unnamed22.notna()].index
bad_index2

Int64Index([1478], dtype='int64')

In [35]:
df.Unnamed22[1478] = 0
df.Unnamed22=df.Unnamed22.fillna(0)

In [36]:
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

Case_Number       1
Fatal           539
Time           3354
dtype: int64

In [37]:
#df.to_excel('/home/jbloch3/Documents/prueba.xlsx', index=False)

In [38]:
#set(df.Date)Zambezi

In [39]:
#df.dtypes

**Normalizamos la variable Date**

**Limpiamos la columna de typos**

In [40]:
df['Date'] = df.Date.apply(lambda x: x.replace('Reported ', '').replace('Before', '').replace('Early ', '')
                           .replace('Late ', '').replace('--',  '-').strip())

In [41]:
#set(df.Date)

**Normalizamos la fecha**

In [42]:
df.Date = pd.to_datetime(df.Date, errors='coerce')



In [43]:
print(df.Date)

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6297   1903-01-01
6298   1903-01-01
6299          NaT
6300          NaT
6301          NaT
Name: Date, Length: 6302, dtype: datetime64[ns]


In [44]:
df.Date = df.Date.dt.strftime('%d/%m/%y')

**Rellenamos na's con fillna con los métodos "bfill y ffill"**

In [45]:
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

Case_Number       1
Date            218
Fatal           539
Time           3354
dtype: int64

In [46]:
df.Date = df.Date.fillna(method='bfill') #Rellenamos con el último valor abservado bueno
df.Date = df.Date.fillna(method='ffill') #Rellenamos con el anterior valor abservado bueno los últimos 3

In [47]:
df.Date = pd.to_datetime(df.Date, errors='coerce')


**Rehacemos la variable Year con los datos de Date**

In [48]:
df['Year'] = df['Date'].dt.year

In [49]:
df

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator_Source,pdf,Href_formula,Href,Case_Number.1,Case_Number.2,Original_order,Unnamed22,Unnamed23
0,2018.06.25,2018-06-25,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,0,0
1,2018.06.18,2018-06-18,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,Unknown,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,0,0
2,2018.06.09,2018-09-06,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,Unknown,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,0,0
3,2018.06.08,2018-08-06,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,0,0
4,2018.06.04,2018-04-06,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,2003-01-01,2003,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,Unknown,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,0,0
6298,ND.0004,2003-01-01,2003,Unprovoked,AUSTRALIA,Western Australia,Unknown,Pearl diving,Ahmun,M,...,Unknown,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,0,0
6299,ND.0003,2003-01-01,2003,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,Unknown,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,0,0
6300,ND.0002,2003-01-01,2003,Unprovoked,PANAMA,Unknown,"Panama Bay 8�N, 79�W",Unknown,Jules Patterson,M,...,Unknown,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,0,0


**Limpiamos la variable Country**

In [50]:
#set(df.Country)


In [51]:
df['Country'] = df.Country.apply(lambda x: x.replace('TURKS & CAICOS', 'TURKEY').replace('SOLOMON ISLANDS / VANUATU', 'SOLOMON ISLANDS')
                                 .replace('ITALY / CROATIA', 'ITALY').replace('EQUATORIAL GUINEA / CAMEROON', 'EQUATORIAL GUINEA')
                                 .replace('TUNISIA', 'TURKEY').replace('?', '').replace('IRAN / IRAQ', 'IRAN').replace('ANDAMAN / NICOBAR ISLANDAS','ANDAMAN ISLANDS')
                                 .replace('EGYPT / ISRAEL', 'EGYPT').replace('CEYLON (SRI LANKA)', 'CEYLON').replace(' (UAE)', '')
                                 .replace('Between PORTUGAL & INDIA', 'PORTUGAL').replace('RED SEA / INDIAN OCEAN', 'RED SEA')
                                 .strip().upper())

**Limpiamos las variables Country, Location, Area, Activity, Name e Injury**

In [52]:
#set(df.Country)

In [53]:
df.Country = df.Country.apply(lambda x: x.strip().upper())

In [54]:
df.Location = df.Location.apply(lambda x: x.strip())

In [55]:
df.Area = df.Area.apply(lambda x: x.strip())

In [56]:
#set(df.Activity)

In [57]:
df.Activity = df.Activity.apply(lambda x: x.strip())

In [58]:
#set(df.Name)

In [59]:
#todos ofrecen info. Borro info del sexo que tenemos en la siguiente columna
df.Name = df.Name.apply(lambda x: x.strip().strip("fe").strip("_").replace('male','Unknown')
                        .replace('female','Unknown').replace(',',''))

In [60]:
#set(df.Injury)

In [61]:
df.Injury = df.Injury.apply(lambda x: x.strip().strip(',').replace(',','').strip(','))

In [62]:
set(df.Fatal)

{' N', '2017', 'M', 'N', 'N ', 'UNKNOWN', 'Y', nan, 'y'}

In [63]:
df['Fatal'] = df['Fatal'].astype('str') 

In [64]:
set(df.Fatal)

{' N', '2017', 'M', 'N', 'N ', 'UNKNOWN', 'Y', 'nan', 'y'}

**Correjimos con replace las variables mal escritas**

In [65]:
df.Fatal = df.Fatal.apply(lambda x: x.strip().replace('2017','Unknown').replace('nan','Unknown').replace('M','N')
                          .replace('UNKNOWN','Unknown').replace('y','Y'))

df['Fatal']=df['Fatal'].fillna('Unknown')
        

In [66]:
set(df.Fatal)

{'N', 'Unknown', 'Y'}

**Hacemnos un indice de los datos Unknown de Fatal y los printeamos con Injury**

In [67]:
ind = df[df["Fatal"]=='Unknown'].index.values

unknown_fatal = df[(df['Fatal']=='Unknown')]
unknown_fatal = unknown_fatal[['Injury', 'Fatal']]
unknown_fatal

,Injury,Fatal
27,Multiple severe injuries to arms and leg leg s...,Unknown
76,FATAL,Unknown
79,Lacerations to right ankle foot & toe,Unknown
84,FATAL but death was probably due to drowning,Unknown
98,Lacerations to left foot,Unknown
...,...,...
6169,Leg severed mid-thigh hand severed arm above e...,Unknown
6171,Unknown,Unknown
6172,Unknown,Unknown
6232,A 4.3 m [14'] shark made threat display. No in...,Unknown


**Habiendo leido todas las palbras clave que prueban muerte en set(df.Injury[ind]), las utilizamos para completar la columna fatal con información de Injury**

In [68]:
#si una de las palabras del fatality_causes está en en los injuries, asumimos que ha muerto
fatality_causes = ["fatal", "body", "remains", "washed", "killed", "perished", "drowned"
                   , "drowning", "bodies", "drowning", "death", "died", "death"]
for i in ind:
    for j in fatality_causes:
        if j in df.Injury[i].lower():
            df.Fatal[i] = "Y"
        elif df.Fatal[i] == "Unknown":
            pass
        elif j not in df.Injury[i].lower():
            df.Fatal[i] = "N"

In [69]:
set(df.Fatal)

{'N', 'Unknown', 'Y'}

**Corregimos los valores N y Y**

In [70]:
for i in range(len(df)):
    if df.Fatal[i] == "Y":
        df.Fatal[i] = True
    elif df.Fatal[i] == "N":
        df.Fatal[i] = False
    else:
        pass



In [71]:
set(df.Fatal)

{False, True, 'Unknown'}

**Cambiamos valores restantes con strings por False(asumimos que sobrevivieron)(son pocos datos)**

In [72]:
ft = [True, False, "Unknown"]
for i in range(len(df)):
    if df.Fatal[i] not in ft:
        df.Fatal[i] = "False" 

In [73]:
df.Fatal[4:15]

4     False
5     False
6      True
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
Name: Fatal, dtype: object

**Normalizamos la variable Time:**

In [74]:
df.dtypes

Case_Number                    object
Date                   datetime64[ns]
Year                            int64
Type                           object
Country                        object
Area                           object
Location                       object
Activity                       object
Name                           object
Sex                            object
Age                             int64
Injury                         object
Fatal                          object
Time                           object
Species                        object
Investigator_Source            object
pdf                            object
Href_formula                   object
Href                           object
Case_Number.1                  object
Case_Number.2                  object
Original_order                float64
Unnamed22                       int64
Unnamed23                       int64
dtype: object

In [75]:
df.Time = df.Time.astype(str)

**Sustituimos algunas palablas claves por la hora correspondiente:**

In [76]:
df['Time'] = df.Time.apply(lambda x: x.replace('Morning', '06h00').replace('Dark', '10h00').replace('Dawn', '06h30')
                           .replace('Dusk', '20h00').replace('Afternoon', '15h00').replace('Midday', '12h00')
                           .replace('Noon', '12h00').replace('Midnight', '00h00').replace('Night', '12h00')
                           .replace('Sundown', '20h00').replace('morning', '06h00').replace('dark', '10h00').replace('Dawn', '06h30')
                           .replace('dusk', '20h00').replace('afternoon', '15h00').replace('midday', '12h00')
                           .replace('noon', '12h00').replace('midnight', '00h00').replace('night', '12h00')
                           .replace('sundown', '20h00'))

**Buscamos el patron y sustiyuimos la columna por la lista de patrones. si no hay ponemos la hora media buscada en:**
(https://underwater.com.au/article/id/10214-shark-attacks-7-answers-to-frequently-asked-questions/#:~:text=Most%20attacks%20world%2Dwide%20happen,spending%20time%20in%20the%20water.)
 

In [77]:
patron=r'\d\dh\d\d'
lst= []
for e in df.Time:
    try:
        lst.append(re.findall(patron,e)[0])
    except:
        lst.append("13h00")

df.Time=lst    

In [78]:
df['Time']


0       18h00
1       14h00
2       07h45
3       13h00
4       13h00
        ...  
6297    13h00
6298    13h00
6299    13h00
6300    13h00
6301    13h00
Name: Time, Length: 6302, dtype: object

In [79]:
df['Time'] = df.Time.apply(lambda x: x.replace('h', ':'))

In [80]:
df['Time']



0       18:00
1       14:00
2       07:45
3       13:00
4       13:00
        ...  
6297    13:00
6298    13:00
6299    13:00
6300    13:00
6301    13:00
Name: Time, Length: 6302, dtype: object

**Intento de ponerlo en formato datetime:**

In [81]:
df.Time = pd.to_datetime(df.Time, format= '%H:%M')

In [82]:
df['Time']

0      1900-01-01 18:00:00
1      1900-01-01 14:00:00
2      1900-01-01 07:45:00
3      1900-01-01 13:00:00
4      1900-01-01 13:00:00
               ...        
6297   1900-01-01 13:00:00
6298   1900-01-01 13:00:00
6299   1900-01-01 13:00:00
6300   1900-01-01 13:00:00
6301   1900-01-01 13:00:00
Name: Time, Length: 6302, dtype: datetime64[ns]

In [83]:
df['Time'].dtypes

dtype('<M8[ns]')

In [84]:
df.Time = df.Time.dt.strftime('%H:%M')

#df.Time = pd.to_datetime(df.Time, format= '%H:%M')



In [85]:
df.Time

0       18:00
1       14:00
2       07:45
3       13:00
4       13:00
        ...  
6297    13:00
6298    13:00
6299    13:00
6300    13:00
6301    13:00
Name: Time, Length: 6302, dtype: object

In [86]:
df['Time'].dtypes

dtype('O')

**Hacemos una lista de especies sacadas de la columna, y hacemos una columna nueva con las especies limpias**

In [87]:
#set(df["Species"])

In [88]:
shark_species = ["white", "carpet", "zambesi", "banjo", "blue", "grey", "wobbegong", "blacktip", "raggedtooth", "wobbegong", "salmon", "raggedtooth", "silky", "metre", "shovelnose", "nurse", "basking", "hammerhead", "blackfin", "blacktip", "port jackson", "sevengill", "whaler", "lemon", "blacktip", "tiger", "caribbean", "spinner", "dusky", "sandbar", "sandtiger", "smoothhound", "bronze"]
for i in range(len(df)):
    for j in shark_species:
        if j in df.Species[i].lower():
            df.Unnamed22[i] = j.capitalize() 
            break
        else:
            df.Unnamed22[i] = "Undetermined"

**Cambiamos los nombres de las colunas y las ordenamos**

In [89]:
df = df.rename(columns = {'Unnamed22': 'Sark_species', 'Species': 'Shark_info'})

In [90]:
df.columns

Index(['Case_Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time',
       'Shark_info', 'Investigator_Source', 'pdf', 'Href_formula', 'Href',
       'Case_Number.1', 'Case_Number.2', 'Original_order', 'Sark_species',
       'Unnamed23'],
      dtype='object')

In [91]:
df = df[['Case_Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time',
       'Shark_info', 'Sark_species', 'Investigator_Source', 'pdf', 'Href_formula', 'Href',
       'Case_Number.1', 'Case_Number.2', 'Original_order',
       'Unnamed23']]

**Reacemos la columna Case_Number con un índice especial(hay una copia en Case_Number.2)**

In [92]:
for i in range(len(df)):
    
    num = str(6302-int(i))
    
    if i < 5303:       
        df.Case_Number[i] = "ND."+ num
    elif i < 6203:
        df.Case_Number[i] = "ND.0"+num
    elif i< 6293:
        df.Case_Number[i] = "ND.00"+num
    elif i < 6302:
        df.Case_Number[i] = "ND.000"+num

In [93]:
df.Case_Number

0       ND.6302
1       ND.6301
2       ND.6300
3       ND.6299
4       ND.6298
         ...   
6297    ND.0005
6298    ND.0004
6299    ND.0003
6300    ND.0002
6301    ND.0001
Name: Case_Number, Length: 6302, dtype: object

**Hemos acabado con los na:**

In [94]:
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

Series([], dtype: int64)

In [95]:
df.head()

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Shark_info,Sark_species,Investigator_Source,pdf,Href_formula,Href,Case_Number.1,Case_Number.2,Original_order,Unnamed23
0,ND.6302,2018-06-25,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wol,F,...,White shark,White,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,0
1,ND.6301,2018-06-18,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,Unknown,Undetermined,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,0
2,ND.6300,2018-09-06,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,Unknown,Undetermined,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,0
3,ND.6299,2018-08-06,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,mal,M,...,2 m shark,Undetermined,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,0
4,ND.6298,2018-04-06,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",Tiger,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,0


In [98]:
df.to_csv("/home/jbloch3/Documents/Proyecto_TL/clean_attacks.csv", index=False)